In [1]:
import pandas as pd

from langchain.chat_models import init_chat_model
from typing import Optional, List, Dict
from typing_extensions import Annotated, TypedDict
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.tools import tool

/data/dania.moriazi01/anaconda3/envs/space/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
df = pd.read_csv('entities.csv')
df.head()

,title,author,id,data,code_link,packages_used,task,field,abstract,models,...,methodology,metrics,limitations,reproducibility,strengths,weaknesses,reuse_potential,impact,metrics_list,scores
0,Generation of Modern Satellite Data from Galil...,"Harim Lee, Eunsu Park, Yong-Jae Moon",DOI: 10.3847/1538-4357/abd498,['Mount Wilson Observatory (MWO) sunspot drawi...,None provided,"['NumPy', 'Keras', 'TensorFlow', 'SunPy']",Generation of modern satellite data (magnetogr...,"Solar Physics (Sunspots, Solar Magnetic Fields...",This study presents a deep learning-based appr...,['Conditional Generative Adversarial Network (...,...,"The study uses a pix2pix model, a type of cond...",['Total Unsigned Magnetic Flux (TUMF) Correlat...,"[""The model does not successfully generate act...",4,['Innovative approach to bridge historical sun...,['Limited ability to reproduce detailed corona...,['The methodology can be extended to other his...,8,[{'Total Unsigned Magnetic Flux (TUMF) Correla...,"[0.82, 0.74, 0.75, 0.56, 0.65, 0.67, 0.7, 0.76..."
1,Visual Explanation of a Deep Learning Solar Fl...,"Kangwoo Yi, Yong-Jae Moon, Daye Lim, Eunsu Par...",DOI: 10.3847/2041-8213/abe94a,['Full-disk magnetograms from SOHO/MDI and SDO...,None provided,"['PyTorch', 'NumPy', 'Matplotlib']",Solar flare prediction,Solar flares,The study presents a visual explanation of a d...,['Convolutional Neural Network (CNN) with dens...,...,The study uses a CNN-based deep learning model...,"['True Skill Statistics (TSS)', 'Accuracy (ACC...",['Projection effects near the solar limb may a...,3,['First application of guided backpropagation ...,['Limited by the resolution and noise in magne...,['The methodology can be extended to other sol...,8,"[{'True Skill Statistics (TSS)': 0.65}, {'Accu...","[0.65, 0.83, 0.65, 0.61]"
2,Improved AI-generated Solar Farside Magnetogra...,"Hyun-Jin Jeong, Yong-Jae Moon, Eunsu Park, Har...",10.3847/2041-8213/ac911f,"['SDO/AIA EUV images (304, 193, 171 Å)', 'SDO/...",https://github.com/JeongHyunJin/Pix2PixCC,"['PyTorch', 'NumPy', 'Matplotlib', 'SciPy', 'A...",Generation of solar farside magnetograms,Solar Physics,The paper presents an improved artificial inte...,"['Pix2PixCC (Generator, Discriminator, Inspect...",...,"The study introduces the Pix2PixCC model, whic...","['Pixel-to-pixel CC (full disk)', 'Pixel-to-pi...",['Physical quantities based on pixel-to-pixel ...,4,['Improved accuracy in generating solar farsid...,['Dependence on the quality and availability o...,['The AI-generated magnetograms can be used fo...,8,"[{'Pixel-to-pixel CC (full disk)': 0.88}, {'Pi...","[0.88, 0.91, 0.7, 0.99, 0.94, 0.94, 0.9, 0.94,..."
3,Construction of global IGS-3D electron density...,"Eun-Young Ji, Yong-Jae Moon, Young-Sil Kwak, K...",10.1016/j.jastp.2024.106370,['International Global Navigation Satellite Sy...,None provided,['PyTorch'],Global 3-D electron density modeling,Ionospheric electron density,The study presents a deep learning-based metho...,['Multi-Layer Perceptron (MLP)'],...,The study employs a deep learning approach usi...,"['Root Mean Square Error (RMSE) (Jicamarca)', ...",['Dependence on IRI-2016 model data for traini...,4,['Improved accuracy in electron density predic...,['Model performance may be affected by the lim...,['The model can be adapted for real-time globa...,8,[{'Root Mean Square Error (RMSE) (Jicamarca)':...,"[0.37, 0.22, 0.34]"
4,Generation of He I 1083 nm Images from SDO AIA...,"Jihyeon Son, Junghun Cha, Yong-Jae Moon, Harim...",10.3847/1538-4357/ac133e,"['SDO/AIA 19.3 nm images', 'SDO/AIA 30.4 nm im...",None provided,"['SunPy', 'TensorFlow or PyTorch (implied by d...",Image-to-image translation (generating He I 10...,Solar Physics,The study presents a deep learning-based metho...,['pix2pixHD (conditional Generative Adversaria...,...,"The study utilized the pix2pixHD model, a cond...","['Correlation Coefficient (CC)', 'Root Mean Sq...",['Synthetic images may miss fine details in fi...,3,['Successful generation of He I 1083 nm images..

In [51]:
llm = init_chat_model('mistral-large-latest', model_provider='mistralai')

In [15]:
df.columns

Index(['title', 'author', 'id', 'data', 'code_link', 'packages_used', 'task',
       'field', 'abstract', 'models', 'hybrid_model', 'multimodal',
       'baselines', 'preprocessing', 'citations', 'approach_used',
       'methodology', 'metrics', 'limitations', 'reproducibility', 'strengths',
       'weaknesses', 'reuse_potential', 'impact', 'metrics_list', 'scores'],
      dtype='object')

In [16]:
entities_df = df.drop(columns=['metrics_list'])
entities_df['code_link'] = [code if 'http' in code else None for code in entities_df['code_link']]

In [17]:
entities_df

,title,author,id,data,code_link,packages_used,task,field,abstract,models,...,approach_used,methodology,metrics,limitations,reproducibility,strengths,weaknesses,reuse_potential,impact,scores
0,Generation of Modern Satellite Data from Galil...,"Harim Lee, Eunsu Park, Yong-Jae Moon",DOI: 10.3847/1538-4357/abd498,['Mount Wilson Observatory (MWO) sunspot drawi...,None,"['NumPy', 'Keras', 'TensorFlow', 'SunPy']",Generation of modern satellite data (magnetogr...,"Solar Physics (Sunspots, Solar Magnetic Fields...",This study presents a deep learning-based appr...,['Conditional Generative Adversarial Network (...,...,['Image-to-image translation using pix2pix (cG...,"The study uses a pix2pix model, a type of cond...",['Total Unsigned Magnetic Flux (TUMF) Correlat...,"[""The model does not successfully generate act...",4,['Innovative approach to bridge historical sun...,['Limited ability to reproduce detailed corona...,['The methodology can be extended to other his...,8,"[0.82, 0.74, 0.75, 0.56, 0.65, 0.67, 0.7, 0.76..."
1,Visual Explanation of a Deep Learning Solar Fl...,"Kangwoo Yi, Yong-Jae Moon, Daye Lim, Eunsu Par...",DOI: 10.3847/2041-8213/abe94a,['Full-disk magnetograms from SOHO/MDI and SDO...,None,"['PyTorch', 'NumPy', 'Matplotlib']",Solar flare prediction,Solar flares,The study presents a visual explanation of a d...,['Convolutional Neural Network (CNN) with dens...,...,['Visual explanation using guided backpropagat...,The study uses a CNN-based deep learning model...,"['True Skill Statistics (TSS)', 'Accuracy (ACC...",['Projection effects near the solar limb may a...,3,['First application of guided backpropagation ...,['Limited by the resolution and noise in magne...,['The methodology can be extended to other sol...,8,"[0.65, 0.83, 0.65, 0.61]"
2,Improved AI-generated Solar Farside Magnetogra...,"Hyun-Jin Jeong, Yong-Jae Moon, Eunsu Park, Har...",10.3847/2041-8213/ac911f,"['SDO/AIA EUV images (304, 193, 171 Å)', 'SDO/...",https://github.com/JeongHyunJin/Pix2PixCC,"['PyTorch', 'NumPy', 'Matplotlib', 'SciPy', 'A...",Generation of solar farside magnetograms,Solar Physics,The paper presents an improved artificial inte...,"['Pix2PixCC (Generator, Discriminator, Inspect...",...,['Deep learning-based image-to-image translati...,"The study introduces the Pix2PixCC model, whic...","['Pixel-to-pixel CC (full disk)', 'Pixel-to-pi...",['Physical quantities based on pixel-to-pixel ...,4,['Improved accuracy in generating solar farsid...,['Dependence on the quality and availability o...,['The AI-generated magnetograms can be used fo...,8,"[0.88, 0.91, 0.7, 0.99, 0.94, 0.94, 0.9, 0.94,..."
3,Construction of global IGS-3D electron density...,"Eun-Young Ji, Yong-Jae Moon, Young-Sil Kwak, K...",10.1016/j.jastp.2024.106370,['International Global Navigation Satellite Sy...,None,['PyTorch'],Global 3-D electron density modeling,Ionospheric electron density,The study presents a deep learning-based metho...,['Multi-Layer Perceptron (MLP)'],...,"['Deep learning-based inversion method', 'Mult...",The study employs a deep learning approach usi...,"['Root Mean Square Error (RMSE) (Jicamarca)', ...",['Dependence on IRI-2016 model data for traini...,4,['Improved accuracy in electron density predic...,['Model performance may be affected by the lim...,['The model can be adapted for real-time globa...,8,"[0.37, 0.22, 0.34]"
4,Generation of He I 1083 nm Images from SDO AIA...,"Jihyeon Son, Junghun Cha, Yong-Jae Moon, Harim...",10.3847/1538-4357/ac133e,"['SDO/AIA 19.3 nm images', 'SDO/AIA 30.4 nm im...",None,"['SunPy', 'TensorFlow or PyTorch (implied by d...",Image-to-image translation (generating He I 10...,Solar Physics,The study presents a deep learning-based metho...,['pix2pixHD (conditional Generative Adversaria...,...,['Image-to-image translation using pix2pixHD.'...,"The study utilized the pix2pixHD model, a cond...","['Correlation Coefficient (CC)', 'Root Mean Sq...",['Synthetic images may miss fine details in fi...,3,['Successful generation of He I 108

In [18]:
class AuthorRelation(TypedDict):
    wrote: Annotated[List[str], ..., 'Papers author wrote or contributed to']

In [40]:
authors = []
for author in entities_df['author']:
    auths = author.split(',')
    auths = [auth.strip() for auth in auths]
    auths = [auth.replace('and', "").strip() if 'and' in auth else auth for auth in auths]
    authors.extend(auths)

In [74]:
entities = []

for ent in entities_df.index:
    ents = {}
    for i, col in enumerate(entities_df):
        ents[col] = df.iloc[ent, i]
    entities.append(ents)

In [78]:
system_prompt = f"""Extract the papers the given author wrote or contributed to from the dataset given.

Dataset: {entities}"""

In [79]:
prompt = ChatPromptTemplate.from_messages=([
    ('system', system_prompt),
    ('human', 'Author: {author}')
])

In [ ]:
class KB():
    def __init__(self):
        self.entities = {}
        self.relations = []

        